In [ ]:
from easyAI import TwoPlayerGame, Human_Player, AI_Player, Negamax, SSS
from random import randint
import pandas as pd
import time


In [6]:
class Piles:
    def __init__(self, piles):
        self.piles = piles.copy()

    def is_pile_empty(self, pile):
        return self.piles[pile] == 0

    def take_from_pile(self, pile, n):
        if n <= self.piles[pile]:
            self.piles[pile] -= n
            return True
        else:
            return False

    def is_end_of_game(self):
        return all(p == 0 for p in self.piles)

class Nimby (TwoPlayerGame):

    def __init__(self, players, probabilistic_version):
        self.players = players
        self.board = Piles([1, 3, 5])
        self.current_player = 1
        self.probabilistic_version = probabilistic_version

    def possible_moves(self):
        return ["%d %d" % (pile+1, i) for pile, n in enumerate(self.board.piles) for i in range(1, n + 1)]

    def make_move(self, move):
        pile, count = map(int, move.split())
        if self.probabilistic_version:
            count = count - 1 if randint(1, 10) == 1 else count
        self.board.take_from_pile(pile-1, count)

    def win(self):
        return self.board.is_end_of_game()

    def is_over(self): return self.win()

    def show(self):
        for ix, n in enumerate(self.board.piles):
            print(f"Pile {ix + 1}: {n}")

    def scoring(self): return 100 if self.win() else 0


In [7]:
inf = float("infinity")


def negamax_no_ab(game, depth, origDepth, scoring):
    """
    Prosta implementacja negamax bez alfa-beta prunning.
    Przechodzimy rekurencyjnie przez wszystkie możliwe ruchy.
    """
    if depth == 0 or game.is_over():
        # Bonus 0.001*depth sprawia, że zwycięstwo szybkie jest lepsze.
        return scoring(game) * (1 + 0.001 * depth)

    bestValue = -inf
    best_move = None
    possible_moves = game.possible_moves()
    # Jeśli obiekt gry wspiera cofanie ruchu, możemy modyfikować stan gry "in place"
    unmake_move = hasattr(game, "unmake_move")

    for move in possible_moves:
        if unmake_move:
            game.make_move(move)
            game.switch_player()
            value = -negamax_no_ab(game, depth - 1, origDepth, scoring)
            game.switch_player()
            game.unmake_move(move)
        else:
            # Jeśli brak metody cofania ruchu, tworzymy kopię stanu gry
            game_copy = game.copy()
            game_copy.make_move(move)
            game_copy.switch_player()
            value = -negamax_no_ab(game_copy, depth - 1, origDepth, scoring)

        if value > bestValue:
            bestValue = value
            best_move = move

    # Przy początkowej głębokości zapisujemy najlepszy ruch
    if depth == origDepth:
        game.ai_move = best_move

    return bestValue


class NegamaxNoAlphaBeta:
    def __init__(self, depth, scoring=None):
        self.depth = depth
        self.scoring = scoring

    def __call__(self, game):
        scoring = self.scoring if self.scoring else (lambda g: g.scoring())
        negamax_no_ab(game, self.depth, self.depth, scoring)
        return game.ai_move

In [8]:
def simulate_games(n_games, depth, probabilistic):

    ai = Negamax(depth)

    wins = {1: 0, 2: 0}

    for i in range(n_games):
        game = Nimby([AI_Player(ai), AI_Player(ai)], probabilistic)
        game.current_player = 1 if i % 2 == 0 else 2
        game.play()
        winner = game.opponent_index
        wins[winner] += 1

    return wins

In [9]:
def simulate_games_time(n_games, depth, probabilistic, algorithm):
    if algorithm == "Negamax":
        ai = NegamaxNoAlphaBeta(depth)
    elif algorithm == "Negamax Alpha-Beta":
        ai = Negamax(depth)
    elif algorithm == "SSS*":
        ai = SSS(depth)

    wins = {1: 0, 2: 0}
    total_time = 0
    move_count = 0

    for i in range(n_games):
        game = Nimby([AI_Player(ai), AI_Player(ai)], probabilistic)
        game.current_player = 1 if i % 2 == 0 else 2

        while not game.is_over():
            start_time = time.time()
            game.play_move(game.get_move())
            total_time += time.time() - start_time
            move_count += 1

        winner = game.opponent_index
        wins[winner] += 1

    avg_time = total_time / move_count if move_count > 0 else 0
    return wins, avg_time


In [10]:
n_games = 10

depths = [5, 13]

In [11]:
results1 = []

for depth in depths:
    wins_det = simulate_games(n_games, depth, probabilistic=False)
    results1.append(["Deterministic", depth, wins_det[1], wins_det[2]])

    wins_prob = simulate_games(n_games, depth, probabilistic=True)
    results1.append(["Probabilistic", depth, wins_prob[1], wins_prob[2]])

columns1 = [ "Game Type", "Depth", "Player 1 Wins", "Player 2 Wins"]
df_results1 = pd.DataFrame(results1, columns=columns1)

print(df_results1.to_string(index=False))

df_results1.to_csv("results1.csv", index=False)

Pile 1: 1
Pile 2: 3
Pile 3: 5

Move #1: player 1 plays 1 1 :
Pile 1: 0
Pile 2: 3
Pile 3: 5

Move #2: player 2 plays 3 1 :
Pile 1: 0
Pile 2: 3
Pile 3: 4

Move #3: player 1 plays 3 1 :
Pile 1: 0
Pile 2: 3
Pile 3: 3

Move #4: player 2 plays 2 1 :
Pile 1: 0
Pile 2: 2
Pile 3: 3

Move #5: player 1 plays 3 1 :
Pile 1: 0
Pile 2: 2
Pile 3: 2

Move #6: player 2 plays 2 1 :
Pile 1: 0
Pile 2: 1
Pile 3: 2

Move #7: player 1 plays 3 2 :
Pile 1: 0
Pile 2: 1
Pile 3: 0

Move #8: player 2 plays 2 1 :
Pile 1: 0
Pile 2: 0
Pile 3: 0
Pile 1: 1
Pile 2: 3
Pile 3: 5

Move #1: player 2 plays 1 1 :
Pile 1: 0
Pile 2: 3
Pile 3: 5

Move #2: player 1 plays 3 1 :
Pile 1: 0
Pile 2: 3
Pile 3: 4

Move #3: player 2 plays 3 1 :
Pile 1: 0
Pile 2: 3
Pile 3: 3

Move #4: player 1 plays 2 1 :
Pile 1: 0
Pile 2: 2
Pile 3: 3

Move #5: player 2 plays 3 1 :
Pile 1: 0
Pile 2: 2
Pile 3: 2

Move #6: player 1 plays 2 1 :
Pile 1: 0
Pile 2: 1
Pile 3: 2

Move #7: player 2 plays 3 2 :
Pile 1: 0
Pile 2: 1
Pile 3: 0

Move #8: player 1 plays 

In [12]:
algorithms = ["Negamax", "Negamax Alpha-Beta", "SSS*"]

results2 = []

for algorithm in algorithms:
    for depth in depths:
        wins_det, avg_time_det = simulate_games_time(n_games, depth, probabilistic=False, algorithm=algorithm)
        results2.append([algorithm, depth, "Deterministic", wins_det[1], wins_det[2], avg_time_det])

        wins_prob, avg_time_prob = simulate_games_time(n_games, depth, probabilistic=True, algorithm=algorithm)
        results2.append([algorithm, depth, "Probabilistic", wins_prob[1], wins_prob[2], avg_time_prob])

columns2 = ["Algorithm", "Depth", "Game Type", "Player 1 Wins", "Player 2 Wins", "Avg Time per Move (s)"]
df_results2 = pd.DataFrame(results2, columns=columns2)

print(df_results2.to_string(index=False))

df_results2.to_csv("results2.csv", index=False)

         Algorithm  Depth     Game Type  Player 1 Wins  Player 2 Wins  Avg Time per Move (s)
           Negamax      5 Deterministic              5              5               0.016198
           Negamax      5 Probabilistic              6              4               0.027976
           Negamax     13 Deterministic              5              5               0.046166
           Negamax     13 Probabilistic              5              5               0.284126
Negamax Alpha-Beta      5 Deterministic              5              5               0.004461
Negamax Alpha-Beta      5 Probabilistic              3              7               0.005457
Negamax Alpha-Beta     13 Deterministic              5              5               0.023631
Negamax Alpha-Beta     13 Probabilistic              6              4               0.050220
              SSS*      5 Deterministic              5              5               0.009066
              SSS*      5 Probabilistic              5              5 